Test de connaissance 
BIG DATA


In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\Spark\\spark-2.4.7-bin-hadoop2.7'

In [2]:
# Importation

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark import SparkContext

# Instantiation
spark = SparkSession.builder.master("local").appName("FolksDF").getOrCreate()

creation de tables

In [3]:
l1 = [('07890','Jean Paul Sartre'),('05678','Pierre de Ronsard')]
rdd1 = spark.sparkContext.parallelize(l1)
Author = rdd1.toDF(['aid','name'])
Author.createOrReplaceTempView('AuthorSQL')
Author.show()

+-----+-----------------+
|  aid|             name|
+-----+-----------------+
|07890| Jean Paul Sartre|
|05678|Pierre de Ronsard|
+-----+-----------------+



In [4]:
l2 = [('0001','Lexistentialisme est un humanisme','Philosophie'),('0002','Huis clos. Suivi de Les Mouches','Philosophie'),('0003','Mignonne allons voir si la rose','Poeme'),('0004','Les Amours','Poème') ]
rdd1 = spark.sparkContext.parallelize(l2)
book = rdd1.toDF(['bid','title','category'])
book.createOrReplaceTempView('BookSQL')
book.show()

+----+--------------------+-----------+
| bid|               title|   category|
+----+--------------------+-----------+
|0001|Lexistentialisme ...|Philosophie|
|0002|Huis clos. Suivi ...|Philosophie|
|0003|Mignonne allons v...|      Poeme|
|0004|          Les Amours|      Poème|
+----+--------------------+-----------+



In [5]:
l3 = [('S15','toto','Math'),('S16','popo','Eco'),('S17','fofo','Mécanique') ]
rdd1 = spark.sparkContext.parallelize(l3)
Student = rdd1.toDF(['sid','sname','dept'])
Student.createOrReplaceTempView('StudentSQL')
Student.show()

+---+-----+---------+
|sid|sname|     dept|
+---+-----+---------+
|S15| toto|     Math|
|S16| popo|      Eco|
|S17| fofo|Mécanique|
+---+-----+---------+



In [6]:
l4 = [('07890','0001'),('07890','0002'),('05678','0003'),('05678','0003')]
rdd1 = spark.sparkContext.parallelize(l4)
write= rdd1.toDF(['aid','bid'])
write.createOrReplaceTempView('WriteSQL')
write.show()

+-----+----+
|  aid| bid|
+-----+----+
|07890|0001|
|07890|0002|
|05678|0003|
|05678|0003|
+-----+----+



In [37]:
l5 = [('S15','0003','02-01-2020','01-02-2020'),('S15','0002','13-06-2020','null'),
      ('S15','0001','13-06-2020','13-10-2020'),('S16','0002','24-01-2020','24-01-2020'),
     ('S17','0001','12-04-2020','01-07-2020')]
rdd1 = spark.sparkContext.parallelize(l5)
borrow = rdd1.toDF(['sid','bid','checkout-time','return-time'])
borrow.createOrReplaceTempView('borrowSQL')
borrow.show()

+---+----+-------------+-----------+
|sid| bid|checkout-time|return-time|
+---+----+-------------+-----------+
|S15|0003|   02-01-2020| 01-02-2020|
|S15|0002|   13-06-2020|       null|
|S15|0001|   13-06-2020| 13-10-2020|
|S16|0002|   24-01-2020| 24-01-2020|
|S17|0001|   12-04-2020| 01-07-2020|
+---+----+-------------+-----------+



1-Trouver les titres de tous les livres que l'étudiant sid='S15' a emprunté.

In [8]:
# SQL
spark.sql(''' select borrowSQL.bid, title
              from borrowSQL 
              join bookSQL 
              on borrowSQL.bid = bookSQL.bid 
              where borrowSQL.sid='S15' ''').show()

+----+--------------------+
| bid|               title|
+----+--------------------+
|0002|Huis clos. Suivi ...|
|0003|Mignonne allons v...|
|0001|Lexistentialisme ...|
+----+--------------------+



In [9]:
# DSL
borrow.join(book,['bid']) \
          .select('bid','title') \
          .filter(F.col('sid')=='S15') \
          .show()

+----+--------------------+
| bid|               title|
+----+--------------------+
|0002|Huis clos. Suivi ...|
|0003|Mignonne allons v...|
|0001|Lexistentialisme ...|
+----+--------------------+



2-Trouver les titres de tous les livres qui n'ont jamais été empruntés par un étudiant.

In [10]:
# SQL
spark.sql(''' select  title
              from bookSQL 
              LEFT JOIN borrowSQL 
              on bookSQL.bid=borrowSQL.bid 
              where borrowSQL.bid is NULL''').show()

+----------+
|     title|
+----------+
|Les Amours|
+----------+



In [11]:
# DSL
book.join(borrow, 'bid', how="left")\
     .select('title')\
     .filter(F.col('checkout-time').isNull())\
     .show()         

+----------+
|     title|
+----------+
|Les Amours|
+----------+



3-Trouver tous les étudiants qui ont emprunté le livre bid=’0002’

In [12]:
# SQL

spark.sql(''' select borrowSQL.sid, sname, bid
              from borrowSQL 
              join StudentSQL 
              on borrowSQL.sid = StudentSQL.sid 
              where bid='0002' 
              ''').show()

+---+-----+----+
|sid|sname| bid|
+---+-----+----+
|S16| popo|0002|
|S15| toto|0002|
+---+-----+----+



In [13]:
borrow.join(Student,['sid']) \
          .select('sid','sname','bid') \
          .filter(F.col('bid')=='0002') \
          .show()

+---+-----+----+
|sid|sname| bid|
+---+-----+----+
|S16| popo|0002|
|S15| toto|0002|
+---+-----+----+




4-Trouver les titres de tous les livres empruntés par des étudiants en Mécanique (département Mécanique) (ERRATUM !!)

In [14]:
spark.sql(''' select borrowSQL.bid, title, StudentSQL.sid, sname, StudentSQL.dept
              from borrowSQL 
              left join bookSQL 
              on borrowSQL.bid = bookSQL.bid 
              left join StudentSQL
              on borrowSQL.sid=StudentSQL.sid
              where StudentSQL.dept='Mécanique' ''').show()

+----+--------------------+---+-----+---------+
| bid|               title|sid|sname|     dept|
+----+--------------------+---+-----+---------+
|0001|Lexistentialisme ...|S17| fofo|Mécanique|
+----+--------------------+---+-----+---------+



In [15]:
# DSL
borrow.join(book,['bid']) \
          .select('bid','title','sid') \
          .join(Student,['sid'])\
          .filter(F.col('dept')=='Mécanique')\
          .show()

+---+----+--------------------+-----+---------+
|sid| bid|               title|sname|     dept|
+---+----+--------------------+-----+---------+
|S17|0001|Lexistentialisme ...| fofo|Mécanique|
+---+----+--------------------+-----+---------+



5-Trouver les étudiants qui n’ont jamais emprunté de livre.

In [16]:
# SQL

spark.sql(''' select StudentSQL.sid, sname
              from StudentSQL 
              JOIN borrowSQL 
              on StudentSQL.sid=borrowSQL.sid 
              where borrowSQL.bid is NULL''').show()
#tous les étudients de la table Student ont emprunté des livres 

+---+-----+
|sid|sname|
+---+-----+
+---+-----+



In [17]:
# DSL
Student.join(borrow, 'sid', how="left")\
     .select('sid', 'sname')\
     .filter(F.col('checkout-time').isNull())\
     .show() 

#tous les étudients de la table Student ont emprunté des livres 

+---+-----+
|sid|sname|
+---+-----+
+---+-----+



6- Déterminer l’auteur qui a écrit le plus de livres. 

In [18]:
#SQL
# group by name 
spark.sql(''' select count(distinct title) as nbL, name
              from bookSQL 
              join writeSQL 
              on bookSQL.bid = writeSQL.bid 
              join AuthorSQL 
              on writeSQL.aid = AuthorSQL.aid
              group by name
              order by nbL desc''').first()

Row(nbL=2, name='Jean Paul Sartre')

In [19]:
#DSL
nb_livre = write\
            .join(Author,['aid'])\
            .groupby('name')\
            .agg(F.count('aid').alias('nb_livre'))
maximum = nb_livre.collect()[0]['nb_livre']
nb_livre\
    .filter(F.col('nb_livre')==maximum)\
    .show()





+-----------------+--------+
|             name|nb_livre|
+-----------------+--------+
| Jean Paul Sartre|       2|
|Pierre de Ronsard|       2|
+-----------------+--------+



7- Déterminer les personnes qui n’ont pas encore rendu les livres.

In [20]:
# SQL
spark.sql("""select sname
         from  borrowSQL
         join StudentSQL
        on StudentSQL.sid = borrowSQL.sid 
        where borrowSQL.`return-time`= 'null' """).show()

+-----+
|sname|
+-----+
| toto|
+-----+



In [21]:
#DSL
Student.join(borrow, 'sid').select('sname').filter(F.col('return-time')=='null').show()

+-----+
|sname|
+-----+
| toto|
+-----+



8- Créer une nouvelle colonne dans la table borrow qui prend la valeur 1, si la durée d'emprunt est supérieur à 3 mois,  sinon 0.  (utiliser la fonction spark to_date) par la suite faire un export des données en CSV. dans un répertoire nommé contention (Attention pas de path en dur ! )

In [22]:
borrow = borrow\
    .withColumn("format",F.lit("dd-MM-yyyy"))\
    .withColumn("start",F.expr("to_date(`checkout-time`, format)"))\
    .withColumn("end",F.expr("to_date(`return-time`, format)"))\
    .withColumn("plus que 3 mois", 
                F.when(F.months_between(F.col('end'),F.col('start'))>=3,1).\
                otherwise(0))\
    .drop('format','start','end').show()

+---+----+-------------+-----------+---------------+
|sid| bid|checkout-time|return-time|plus que 3 mois|
+---+----+-------------+-----------+---------------+
|S15|0003|   02-01-2020| 01-02-2020|              0|
|S15|0002|   13-06-2020|       null|              0|
|S15|0001|   13-06-2020| 13-10-2020|              1|
|S16|0002|   24-01-2020| 24-01-2020|              0|
|S17|0001|   12-04-2020| 01-07-2020|              0|
+---+----+-------------+-----------+---------------+



9-déterminer les livres qui n’ont jamais été empruntés. 

In [23]:
#SQL
spark.sql("""
    select bookSQL .title
    from bookSQL 
    full join borrowSQL 
    on bookSQL.bid = borrowSQL.bid
    where borrowSQL.sid is NULL""").show()

+----------+
|     title|
+----------+
|Les Amours|
+----------+



In [41]:
# DSL
borrow.join(book, book.bid==borrow.bid, how='right')\
    .select('title')\
    .filter(F.col('sid').isNull())\
    .show()

+----------+
|     title|
+----------+
|Les Amours|
+----------+

